In [2]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
from bo_methods_lib.bo_methods_lib.bo_functions_generic import gen_theta_set, clean_1D_arrays
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
import pickle

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
#Open Data File
# DateTime = "2023/09/01/"
DateTime = "2023/09/04/"
save_csv = False
name_cs_str = "CS1"      
methods = ["A1", "B1", "A2", "B2", "C2"]
ep_methods = ["CONSTANT", "DECAY", "BOYLE", "JASRASARIA"]

#Get theta dimensions from any file
path = DateTime + "Data_Files/" + name_cs_str + "_BO_method_" + methods[0] + "_ep_method_" + ep_methods[0]
fileObj = open(path + ".pickle", 'rb')
results = pickle.load(fileObj)        
theta_dim = results[0].configuration["Number of Parameters"]
fileObj.close()

#Initialize overall min params
sse_min_all = np.ones(len(methods))*np.inf
sse_min_act_all = np.ones(len(methods))*np.inf
theta_min_all = np.zeros((len(methods),theta_dim))
run_all = np.zeros(len(methods))
iter_all = np.zeros(len(methods))
best_ep_meth = ["None"]*len(methods)
    
#Loop over methods
for i in range(len(methods)):
    #Loop over ep_methods
    for j in range(len(ep_methods)):
        #Pull out file
        path = DateTime + "Data_Files/" + name_cs_str + "_BO_method_" + methods[i] + "_ep_method_" + ep_methods[j]
        fileObj = open(path + ".pickle", 'rb')
        results = pickle.load(fileObj)        
        param_names = results[0].simulator_class.theta_true_names
        tot_runs = results[0].configuration["Number of Workflow Restarts"]
        #Loop over runs
        for k in range(tot_runs):
            run_results = results[k].results_df
            #Find lowest sse and corresponding theta
            min_sse_index = np.argmin(run_results['Min Obj'])
            min_sse = run_results['Min Obj'].iloc[min_sse_index]
            min_sse_act = run_results['Min Obj Act'].iloc[min_sse_index]
            min_sse_theta = run_results['Theta Min Obj'].iloc[min_sse_index]
            
            if min_sse_act < sse_min_all[i]:
                sse_min_all[i] = min_sse
                sse_min_act_all[i] = min_sse_act
                theta_min_all[i,:] = min_sse_theta
                iter_all[i] = min_sse_index + 1 #Plus one to start count at 1 and not 0
                run_all[i] = k + 1 #Plus one to start count at 1 and not 0
                best_ep_meth[i] = ep_methods[j]
        #Close file
        fileObj.close()
    #Pandas dataframe of the lowest sse overall and corresponding theta
    column_names = ['BO Method', 'Best EP Method', 'Min SSE', 'Min SSE Act'] + [f'{param_names[i]}' for i in range(theta_dim)] + ['BO Restart', 'BO Iter']
    data = {'BO Method': methods,
        'Best EP Method': best_ep_meth,
        'Min SSE': sse_min_all,
        'Min SSE Act': sse_min_act_all}
    for i in range(theta_dim):
        data[param_names[i]] = theta_min_all[:, i]
    data['BO Restart'] = run_all
    data['BO Iter'] = iter_all
    EP_Analysis = pd.DataFrame(data, columns=column_names)
    #Save CSV Data
    
if save_csv == True:
    path_to_save_df = DateTime + "Exploration_Bias_Exp.csv"
    EP_Analysis.to_csv(path_to_save_df, index=False)
else:
    print(EP_Analysis)
    print(results[0].simulator_class.theta_true)

  BO Method Best EP Method  Min SSE  Min SSE Act         A_1         A_2  \
0        B1       CONSTANT      1.0  9190.807249 -238.461538 -164.102564   

          A_3        A_4  BO Restart  BO Iter  
0 -216.666667  15.384615         1.0      1.0  
[-200. -100. -170.   15.]
